## Задание № 6

### Урок 6. Градиентный бустинг (AdaBoost)

#### Задание

1. Для реализованной в методичке модели градиентного бустинга построить графики зависимости ошибки от количества деревьев в ансамбле и от максимальной глубины деревьев. Сделать выводы о зависимости ошибки от этих параметров.
2. **(*)** Модифицировать реализованный алгоритм, чтобы получился стохастический градиентный бустинг. Размер подвыборки принять равным 0.5. Сравнить на одном графике кривые изменения ошибки на тестовой выборке в зависимости от числа итераций.
3. **(*)** Реализовать алгоритм градиетного бустинга, основанный на реализации решающего дерева из ДЗ4 / методички к уроку 4. Сделать выводы о качестве алгоритма по сравнению с реализацией из п.1.

Напишем функцию, реализующую предсказание в градиентном бустинге.

In [33]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection
import numpy as np

In [34]:
class GBGradBoost:
    n_trees = 0
    max_depth = 0
    eta = 0
    tree_list = []
    train_errors = []
    test_errors = []
    coefs =[]


    def params(self, n_trees=10, max_depth=3, eta=1):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.eta = eta
        self.coefs = [1] * n_trees

    # def __init__(self, X, y, test_size=0.25):
    def __init__(self):
        self.X_train, self.X_test, self.y_train, self.y_test = \
            model_selection.train_test_split(X, y, test_size=test_size)
        self.params()

    def gb_predict(self, X, coef_list, eta):
        # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
        # поэтому все деревья из списка trees_list уже являются дополнительными и при предсказании прибавляются с шагом eta
        return np.array([sum([eta* coef * alg.predict([x])[0] for alg, coef in zip(self.tree_list, coef_list)]) for x in X])


    # В качестве функционала ошибки будем использовать среднеквадратичную ошибку. Реализуем соответствующую функцию.
    def mean_squared_error(self, y_real, prediction):
        return (sum((y_real - prediction) ** 2)) / len(y_real)

    def func_loss(self, y_real, prediction):
        pass

    def bias(self, y, z):
        return (y - z)

    def grad(self, y, z):
        pass

    # Реализуем функцию обучения градиентного бустинга.
    # def gb_fit(self, n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta):
    def gb_fit(self):
        # Деревья будем записывать в список
        # Будем записывать ошибки на обучающей и тестовой выборке на каждой итерации в список
        self.tree_list = []

        for i in range(self.n_trees):
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=42)
            # инициализируем бустинг начальным алгоритмом, возвращающим ноль,
            # поэтому первый алгоритм просто обучаем на выборке и добавляем в список
            if len(self.tree_list) == 0:
                # обучаем первое дерево на обучающей выборке
                tree.fit(self.X_train, self.y_train)

                train_errors.append(self.mean_squared_error(self.y_train, self.gb_predict(self.X_train, self.coefs)))
                test_errors.append(self.mean_squared_error(self.y_test, self.gb_predict(self.X_test, self.coefs)))
            else:
                # Получим ответы на текущей композиции
                target = self.gb_predict(self.X_train, self.coefs, self.eta)

                # алгоритмы начиная со второго обучаем на сдвиг
                tree.fit(self.X_train, self.bias(self.y_train, target))

                train_errors.append(self.mean_squared_error(self.y_train, self.gb_predict(self.X_train, self.tree_list, self.coefs, self.eta)))
                test_errors.append(self.mean_squared_error(self.y_test, self.gb_predict(self.X_test, self.tree_list, self.coefs, self.eta)))

            trees.append(tree)

        return trees, train_errors, test_errors

    def evaluate_alg(self, X_train, X_test, y_train, y_test, trees, coefs, eta):
        train_prediction = self.gb_predict(X_train, trees, coefs, eta)

        print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
        с шагом {eta} на тренировочной выборке: {self.mean_squared_error(y_train, train_prediction)}')

        test_prediction = self.gb_predict(X_test, trees, coefs, eta)

        print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
        с шагом {eta} на тестовой выборке: {self.mean_squared_error(y_test, test_prediction)}')

Теперь обучим несколько моделей с разными параметрами и исследуем их поведение.

In [35]:
from sklearn.datasets import load_diabetes

In [36]:
X, y = load_diabetes(return_X_y=True)

In [37]:
# Число деревьев в ансамбле
n_trees = 10
# для простоты примем коэффициенты равными 1
coefs = [1] * n_trees
# Максимальная глубина деревьев
max_depth = 3
# Шаг
eta = 1

In [38]:
gb_gboost = GBGradBoost()

trees, train_errors, test_errors = gb_gboost.gb_fit(n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta)
gb_gboost.evaluate_alg(X_train, X_test, y_train, y_test, trees, coefs, eta)


TypeError: __init__() missing 2 required positional arguments: 'X' and 'y'

In [20]:
trees[0]

DecisionTreeRegressor(max_depth=3, random_state=42)